In [2]:
import pandas as pd

df = pd.read_csv(
    r"C:\Users\kalki\OneDrive\Desktop\week2\mobile-banking-reviews-Challenge-week2\notebooks\data\task2_sentiment_themes.csv"
)

df.head()
df.columns


Index(['review_id', 'review_text', 'rating', 'review_date', 'user_name',
       'thumbs_up', 'reply_content', 'bank_code', 'bank_name', 'app_version',
       'source'],
      dtype='object')

In [12]:
import pandas as pd
import psycopg2

# Load final CSV
df_pg = pd.read_csv(r"C:\Users\kalki\OneDrive\Desktop\week2\mobile-banking-reviews-Challenge-week2\notebooks\data\processed\reviews_final.csv")

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="kalkidan111",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Insert unique banks first (use correct columns)
banks = df_pg[['bank_name_x', 'source_x']].drop_duplicates()
for bank_name, source in banks.values:
    cur.execute("""
        INSERT INTO banks (bank_name, app_name)
        VALUES (%s, %s)
        ON CONFLICT (bank_name) DO NOTHING;
    """, (bank_name, source))

conn.commit()  # Commit banks

# Insert reviews with conflict handling
for row in df_pg.itertuples():
    try:
        cur.execute("""
            INSERT INTO reviews (
                review_id, bank_id, review_text, rating, review_date,
                sentiment_label, sentiment_score, source
            )
            VALUES (
                %s,
                (SELECT bank_id FROM banks WHERE bank_name = %s),
                %s, %s, %s, %s, %s, %s
            )
            ON CONFLICT (review_id) DO NOTHING;
        """, (
            row.review_id,
            row.bank_name_x,
            row.review_text,
            row.rating_x,
            row.review_date_x,
            row.sentiment_label,
            row.sentiment_score,
            row.source_x
        ))
    except Exception as e:
        print(f"Skipped row {row.review_id} due to error: {e}")
        conn.rollback()
    else:
        conn.commit()

cur.close()
conn.close()
print("Final CSV inserted into PostgreSQL safely!")


Final CSV inserted into PostgreSQL safely!
